# Last Formation

In [ ]:
import json
import re
from collections import defaultdict

# --- Load properties from JSON file ---
try:
    with open("property_data.json", "r") as f:
        properties_data = json.load(f)
except FileNotFoundError:
    print("Error: 'property_data.json' not found. Please ensure the file exists.")
    properties_data = []
except json.JSONDecodeError:
    print("Error: Could not decode 'property_data.json'. Please check its format.")
    properties_data = []

# --- Helper functions for normalization ---
def normalize_area_name(area_name):
    return str(area_name).replace(" ", "").lower().strip()

def normalize_zone_name(zone_name):
    return str(zone_name).replace(" ", "").lower().strip()

def normalize_facility_name(facility_name):
    return str(facility_name).replace(" ", "_").lower().strip()

def normalize_amenity_name(name):
    return str(name).replace(" ", "_").lower().strip()

def normalize_room_name(room_name):
    return str(room_name).replace(" ", "").lower().strip()

def normalize_property_type_name(prop_type):
    return str(prop_type).lower().strip()

def get_numeric_value(value):
    """Extract integer from strings like '800 sqft', '10 years', etc.
       Returns None if no number is found."""
    if not value:
        return None
    match = re.search(r"\d+", str(value))
    return int(match.group()) if match else None


# --- Dynamically get all unique values from the dataset ---
ALL_AREAS = sorted(
    list(set(normalize_area_name(p.get("Area", "N/A")) for p in properties_data))
)

ALL_ZONES = sorted(
    list(set(normalize_zone_name(p.get("Zone", "N/A")) for p in properties_data))
)

ALL_FACILITIES = []
for p in properties_data:
    if "Facilities" in p and isinstance(p["Facilities"], dict):
        ALL_FACILITIES = sorted(list(p["Facilities"].keys()))
        break

ALL_NEARBY_AMENITIES = []
for p in properties_data:
    if "Nearby_Amenities" in p and isinstance(p["Nearby_Amenities"], dict):
        ALL_NEARBY_AMENITIES = sorted(list(p["Nearby_Amenities"].keys()))
        break

ALL_ROOM_TYPES = sorted(
    list(set(normalize_room_name(p.get("Room_Details", {}).get("Rooms", "N/A")) 
             for p in properties_data if p.get("Room_Details", {}).get("Rooms")))
)

ALL_PROPERTY_TYPES = sorted(
    list(set(normalize_property_type_name(p.get("Room_Details", {}).get("Type", "N/A")) 
             for p in properties_data if p.get("Room_Details", {}).get("Type")))
)



# --- Filtering logic ---
def filter_properties(user_input, field, data):
    filtered_properties = []

    # Map search field to the actual key in properties_data
    data_field_map = {
        "size": "Size_In_Sqft", "carpet": "Carpet_Area_Sqft", "age": "Property_Age", "brokerage": "Brokerage",
        "furnishing": "Furnishing_Status", "amenities": "Number_Of_Amenities", "security": "Security_Deposite", "rent": "Rent_Price",
        "area": "Area", "zone": "Zone", "bedrooms": "Bedrooms", "bathrooms": "Bathrooms", "balcony": "Balcony",
        "floor_no": "Floor_No", "total_floors": "Total_floors_In_Building", "maintenance": "Maintenance_Charge",
        "recommended_for": "Recommended_For", "water_supply": "Water_Supply_Type", "society_type": "Society_Type",
        "road_connectivity": "Road_Connectivity", "facilities": "Facilities", "nearby_amenities": "Nearby_Amenities",
        "room_type": "Room_Details", "property_type": "Room_Details", "id": "property_id"
    }
    data_field = data_field_map.get(field)
    if not data_field:
        return []

    # --- String fields ---
    normalized_user_input = user_input.lower().strip()
    if field in ["brokerage", "furnishing", "maintenance", "recommended_for", "water_supply", "society_type"]:
        filtered_properties = [p for p in data if str(p.get(data_field, "N/A")).lower() == normalized_user_input]

    elif field == "facilities":
        user_facilities = [normalize_facility_name(f) for f in user_input.split(',')]
        filtered_properties = [p for p in data if all(
            normalize_facility_name(k) in user_facilities and v == 1 
            for k, v in p.get("Facilities", {}).items() if k
        )]

    elif field == "nearby_amenities":
        user_amenities = [normalize_amenity_name(f) for f in user_input.split(',')]
        filtered_properties = [p for p in data if all(
            normalize_amenity_name(k) in user_amenities and v == 1 
            for k, v in p.get("Nearby_Amenities", {}).items() if k
        )]

    elif field == "room_type":
        filtered_properties = [p for p in data if normalize_room_name(p.get("Room_Details", {}).get("Rooms", "")) == normalized_user_input]

    elif field == "property_type":
        filtered_properties = [p for p in data if normalize_property_type_name(p.get("Room_Details", {}).get("Type", "")) == normalized_user_input]

    elif field == "area":
        filtered_properties = [p for p in data if normalize_area_name(p.get("Area", "N/A")) == normalize_area_name(user_input)]

    elif field == "zone":
        filtered_properties = [p for p in data if normalize_zone_name(p.get("Zone", "N/A")) == normalize_zone_name(user_input)]

    elif field == "id":
        filtered_properties = [p for p in data if str(p.get(data_field)) == user_input.strip()]

    else:
        try:
            val = get_numeric_value(user_input)
    
            if user_input.startswith("below"):
                filtered_properties = [
                    p for p in data
                    if get_numeric_value(p.get(data_field)) is not None 
                    and get_numeric_value(p.get(data_field)) < val
                ]
            elif user_input.startswith("above"):
                filtered_properties = [
                    p for p in data
                    if get_numeric_value(p.get(data_field)) is not None 
                    and get_numeric_value(p.get(data_field)) > val
                ]
            elif user_input.startswith("between"):
                nums = re.findall(r"\d+", user_input)
                if len(nums) == 2:
                    low, high = int(nums[0]), int(nums[1])
                    filtered_properties = [
                        p for p in data
                        if get_numeric_value(p.get(data_field)) is not None
                        and low <= get_numeric_value(p.get(data_field)) <= high
                    ]
            else:
                filtered_properties = [
                    p for p in data
                    if get_numeric_value(p.get(data_field)) == val
                ]
    
        except Exception:
            return []

    return filtered_properties

# --- Format results ---
def format_property(prop):
    property_id = prop.get('property_id', 'N/A')
    rent_price = prop.get('Rent_Price', 'N/A')
    size = prop.get('Size_In_Sqft', 'Unknown')
    carpet_area = prop.get('Carpet_Area_Sqft', 'Unknown')
    security_deposit = prop.get('Security_Deposite', 'N/A')
    brokerage = prop.get('Brokerage', 'N/A')
    furnishing_status = prop.get('Furnishing_Status', 'N/A')
    amenities = prop.get('Number_Of_Amenities',0)
    age = prop.get('Property_Age', 'Unknown')
    area = prop.get('Area', 'N/A')
    zone = prop.get('Zone', 'N/A')
    bedrooms = prop.get('Bedrooms', 'N/A')
    bathrooms = prop.get('Bathrooms', 'N/A')
    balcony = prop.get('Balcony', 'N/A')
    floor_no = prop.get('Floor_No', 'N/A')
    total_floors = prop.get('Total_floors_In_Building', 'N/A')
    maintenance = prop.get('Maintenance_Charge', 'N/A')
    recommended_for = prop.get('Recommended_For', 'N/A')
    water_supply = prop.get('Water_Supply_Type', 'N/A')
    society_type = prop.get('Society_Type', 'N/A')
    road_connectivity = prop.get('Road_Connectivity', 'N/A')
    facilities_list = [k.replace("_", " ").title() for k, v in prop.get("Facilities", {}).items() if v == 1]
    facilities = ', '.join(facilities_list) if facilities_list else 'None'
    nearby_amenities_list = [k.replace("_", " ").title() for k, v in prop.get("Nearby_Amenities", {}).items() if v == 1]
    nearby_amenities = ', '.join(nearby_amenities_list) if nearby_amenities_list else 'None'
    rooms = prop.get("Room_Details", {}).get("Rooms", "N/A")
    property_type = prop.get("Room_Details", {}).get("Type", "N/A")

    return (
        f"ID: {property_id} | Rent: ₹{rent_price} | Size: {size} sqft | Carpet Area: {carpet_area} sqft\n"
        f"Rooms: {rooms} | Property Type: {property_type} | Bedrooms: {bedrooms} | Bathrooms: {bathrooms} | Balcony: {balcony}\n"
        f"Furnishing: {furnishing_status} | Security Deposit: ₹{security_deposit} | Brokerage: {brokerage}\n"
        f"Amenities: {amenities}\n"
        f"Facilities: {facilities}\n"
        f"Nearby Amenities: {nearby_amenities}\n"
        f"Floor: {floor_no}/{total_floors} | Maintenance: {maintenance} | Recommended For: {recommended_for}\n"
        f"Water Supply: {water_supply} | Society: {society_type} | Road Connectivity: {road_connectivity} km\n"
        f"Age: {age} years | Area: {area} | Zone: {zone}\n"
        f"----------------------------------------"
    )

# --- Chatbot ---
def chatbot_response():
    print("Welcome to Property Search Chatbot!")
    print("\nSearch Options:")
    print(
        "1. Size (sqft)\n2. Carpet Area (sqft)\n3. Age of Property\n4. Brokerage (yes/no)\n5. Property ID (exact match)\n"
        "6. Amenities\n7. Furnishing Status\n8. Security Deposit\n9. Rent Price\n10. Area\n11. Zone\n"
        "12. Bedrooms\n13. Bathrooms\n14. Balcony\n15. Floor Number\n16. Total Floors\n17. Maintenance Charge\n"
        "18. Recommended For\n19. Water Supply Type\n20. Society Type\n21. Road Connectivity\n22. Facilities\n"
        "23. Nearby Amenities\n24. Room Type (e.g., 1 BHK)\n25. Property Type (e.g., Flat)\n"
    )
    print("💡 You can combine multiple search fields using '+'.")
    print("Example: 1+2+3 (Size + Carpet Area + Age of Property)")
    print("Type 'exit' to quit.")

    CATEGORY_OPTIONS = {
    "brokerage": sorted(list(set(str(p.get("Brokerage", "N/A")).lower() for p in properties_data))),
    "furnishing": sorted(list(set(str(p.get("Furnishing_Status", "N/A")).lower() for p in properties_data))),
    "maintenance": sorted(list(set(str(p.get("Maintenance_Charge", "N/A")).lower() for p in properties_data))),
    "recommended_for": sorted(list(set(str(p.get("Recommended_For", "N/A")).lower() for p in properties_data))),
    "water_supply": sorted(list(set(str(p.get("Water_Supply_Type", "N/A")).lower() for p in properties_data))),
    "society_type": sorted(list(set(str(p.get("Society_Type", "N/A")).lower() for p in properties_data))),
    "area": ALL_AREAS,
    "zone": ALL_ZONES,
    "room_type": ALL_ROOM_TYPES,
    "property_type": ALL_PROPERTY_TYPES
    }


    search_map = {
        "1": "size", "2": "carpet", "3": "age", "4": "brokerage", "5": "id", "6": "amenities", "7": "furnishing",
        "8": "security", "9": "rent", "10": "area", "11": "zone", "12": "bedrooms", "13": "bathrooms",
        "14": "balcony", "15": "floor_no", "16": "total_floors", "17": "maintenance", "18": "recommended_for",
        "19": "water_supply", "20": "society_type", "21": "road_connectivity", "22": "facilities", "23": "nearby_amenities",
        "24": "room_type", "25": "property_type"
    }
    
    while True:
        field_numbers = input("\nEnter the number(s) of search fields to combine (e.g., 9+10) or 'exit': ").strip().lower()
        if field_numbers == "exit":
            print("Goodbye!")
            break

        selected_fields = []
        is_valid_input = True
        for num in field_numbers.split('+'):
            num = num.strip()
            if num in search_map:
                selected_fields.append(search_map[num])
            else:
                print(f"❌ Invalid field number: '{num}'. Please use a valid number from the list.")
                is_valid_input = False
                break
        
        if not is_valid_input:
            continue

        combined_filters = {}
        for field in selected_fields:
            if field in CATEGORY_OPTIONS and CATEGORY_OPTIONS[field]:
                options = CATEGORY_OPTIONS[field]
                print(f"\nAvailable options for {field.title()}:")
                for idx, option in enumerate(options, start=1):
                    print(f"{idx}. {option}")
                
                choice = input(f"Enter {field} value (choose number or type value): ").strip().lower()
                
                # If user entered a number, map to option
                if choice.isdigit():
                    choice_idx = int(choice) - 1
                    if 0 <= choice_idx < len(options):
                        user_input = options[choice_idx]
                    else:
                        print("❌ Invalid choice, defaulting to your raw input.")
                        user_input = choice
                else:
                    user_input = choice
            else:
                user_input = input(f"Enter {field} value: ").strip()
            
            combined_filters[field] = user_input



        # Special case for 'id'
        if "id" in combined_filters:
            prop_id = combined_filters["id"]
            prop = next((p for p in properties_data if str(p.get("property_id")) == prop_id), None)
            if prop:
                print("\n✅ Property Found:")
                print(format_property(prop))
            else:
                print("❌ Property not found.")
            continue
        
        results = properties_data
        for field, value in combined_filters.items():
            results = filter_properties(value, field, results)
        
        if not results:
            print("❌ No properties found matching your combined search.")
        else:
            print(f"\n✅ Found {len(results)} properties matching your combined query.")
            grouped_results = defaultdict(list)
            for prop in results:
                property_type = prop.get("Room_Details", {}).get("Type", "Other/Unspecified Type")
                grouped_results[property_type].append(prop)
            
            for prop_type, props in grouped_results.items():
                print(f"\n--- 🏠 Property Type: {str(prop_type).title()} ({len(props)} results) ---")
                for prop in props:
                    print(format_property(prop))

if __name__ == "__main__":
    chatbot_response()


Welcome to Property Search Chatbot!

Search Options:
1. Size (sqft)
2. Carpet Area (sqft)
3. Age of Property
4. Brokerage (yes/no)
5. Property ID (exact match)
6. Amenities
7. Furnishing Status
8. Security Deposit
9. Rent Price
10. Area
11. Zone
12. Bedrooms
13. Bathrooms
14. Balcony
15. Floor Number
16. Total Floors
17. Maintenance Charge
18. Recommended For
19. Water Supply Type
20. Society Type
21. Road Connectivity
22. Facilities
23. Nearby Amenities
24. Room Type (e.g., 1 BHK)
25. Property Type (e.g., Flat)

💡 You can combine multiple search fields using '+'.
Example: 1+2+3 (Size + Carpet Area + Age of Property)
Type 'exit' to quit.



Enter the number(s) of search fields to combine (e.g., 9+10) or 'exit':  10



Available options for Area:
1. ashirwadnagar
2. beltarodi
3. besa
4. bharatwada
5. boriyapura
6. chandrakirannagar
7. dabha
8. dhantoli
9. dharampeth
10. dighori
11. duttawadi
12. gandhibagh
13. ganeshpeth
14. godhni
15. gotalpanjri
16. hingna
17. hudkeswar
18. itwari
19. jaitala
20. jaripatka
21. kalamna
22. kalmeshwar
23. khamla
24. kharbi
25. koradicolony
26. kotewada
27. mahal
28. manewada
29. manishnagar
30. mankapur
31. medicalsquare
32. mihan
33. nandanwan
34. narendranagarextension
35. narivillage
36. narsala
37. omkarnagar
38. parvatinagar
39. pratapnagar
40. rameshwari
41. ramnagar
42. reshimbagh
43. sadar
44. sanmarganagar
45. seminaryhills
46. shatabdisquare
47. sitabuldi
48. somalwada
49. sonegaon
50. tekanaka
51. trimurtinagar
52. vayusenanagar
53. wanadongri
54. wardhmannagar
55. wathoda
56. zingabaitakli


Enter area value (choose number or type value):  1



✅ Found 4 properties matching your combined query.

--- 🏠 Property Type: Independent House (2 results) ---
ID: Property_90 | Rent: ₹20000.0 | Size: Total Area 1200.sqft  sqft | Carpet Area: Carpet Area 0.0 sqft sqft
Rooms: 1 BHK | Property Type: Independent House | Bedrooms: 3 | Bathrooms: 3 | Balcony: 3
Furnishing: Unfurnished | Security Deposit: ₹40000.0 | Brokerage: NO
Amenities: 1
Facilities: Parking
Nearby Amenities: Hospital, Mall, Metro Station, School
Floor: 0/0 | Maintenance: No | Recommended For: 0
Water Supply: Both | Society: Non-Gated | Road Connectivity: 1.0 km
Age: 0 years | Area: ashirwad nagar | Zone: West Zone
----------------------------------------
ID: Property_183 | Rent: ₹20000.0 | Size: Total Area 1200.sqft  sqft | Carpet Area: Carpet Area 0.0 sqft sqft
Rooms: 1 BHK | Property Type: Independent House | Bedrooms: 3 | Bathrooms: 3 | Balcony: 3
Furnishing: Unfurnished | Security Deposit: ₹40000.0 | Brokerage: NO
Amenities: 1
Facilities: Parking
Nearby Amenities: Ho

import json
import re
from collections import defaultdict
from tabulate import tabulate # Make sure tabulate is installed

# --- Load properties from JSON file ---
try:
    with open("property_data.json", "r") as f:
        properties_data = json.load(f)
except FileNotFoundError:
    print("Error: 'property_data.json' not found. Please ensure the file exists.")
    properties_data = []
except json.JSONDecodeError:
    print("Error: Could not decode 'property_data.json'. Please check its format.")
    properties_data = []

# --- Helper functions for normalization ---
def normalize_area_name(area_name):
    return str(area_name).replace(" ", "").lower().strip()

def normalize_zone_name(zone_name):
    return str(zone_name).replace(" ", "").lower().strip()

def normalize_facility_name(facility_name):
    return str(facility_name).replace(" ", "_").lower().strip()

def normalize_amenity_name(name):
    return str(name).replace(" ", "_").lower().strip()

def normalize_room_name(room_name):
    return str(room_name).replace(" ", "").lower().strip()

def normalize_property_type_name(prop_type):
    return str(prop_type).lower().strip()

def get_numeric_value(value):
    """Extract integer from strings like '800 sqft', '10 years', etc.
       Returns None if no number is found."""
    if not value:
        return None
    match = re.search(r"\d+", str(value))
    return int(match.group()) if match else None

# --- Dynamically get all unique values from the dataset ---
ALL_AREAS = sorted(
    list(set(normalize_area_name(p.get("Area", "N/A")) for p in properties_data))
)

ALL_ZONES = sorted(
    list(set(normalize_zone_name(p.get("Zone", "N/A")) for p in properties_data))
)

ALL_FACILITIES = []
for p in properties_data:
    if "Facilities" in p and isinstance(p["Facilities"], dict):
        ALL_FACILITIES = sorted(list(p["Facilities"].keys()))
        break

ALL_NEARBY_AMENITIES = []
for p in properties_data:
    if "Nearby_Amenities" in p and isinstance(p["Nearby_Amenities"], dict):
        ALL_NEARBY_AMENITIES = sorted(list(p["Nearby_Amenities"].keys()))
        break

ALL_ROOM_TYPES = sorted(
    list(set(normalize_room_name(p.get("Room_Details", {}).get("Rooms", "N/A"))
             for p in properties_data if p.get("Room_Details", {}).get("Rooms")))
)

ALL_PROPERTY_TYPES = sorted(
    list(set(normalize_property_type_name(p.get("Room_Details", {}).get("Type", "N/A"))
             for p in properties_data if p.get("Room_Details", {}).get("Type")))
)

# --- Comparison Function ---
def compare_properties_side_by_side(data, property_ids):
    """
    Compare multiple properties side by side in table format.
    """
    selected = [p for p in data if str(p.get("property_id", "")).lower() in property_ids]

    if not selected:
        print("⚠️ No properties found for the given IDs.")
        return

    # Collect all possible comparison keys
    comparison_keys = set()
    for p in selected:
        comparison_keys.update(p.keys())
        if isinstance(p.get("Facilities"), dict):
            comparison_keys.update([f"Facility: {k}" for k in p["Facilities"].keys()])
        if isinstance(p.get("Nearby_Amenities"), dict):
            comparison_keys.update([f"Amenity: {k}" for k in p["Nearby_Amenities"].keys()])
        # Also include nested Room_Details keys
        if isinstance(p.get("Room_Details"), dict):
            comparison_keys.update([f"Room Details: {k}" for k in p["Room_Details"].keys()])

    # Always show Property ID and Rent Price first for property comparison
    display_order = ["property_id", "Rent_Price"]
    remaining_keys = sorted(k for k in comparison_keys if k not in display_order)
    comparison_keys = display_order + remaining_keys

    # Build rows
    rows = []
    for key in comparison_keys:
        row = [key.replace('_', ' ').title()]
        for p in selected:
            value = "N/A"
            if key.startswith("Facility: "):
                fname = key.split(": ", 1)[1]
                value = "✅" if p.get("Facilities", {}).get(fname) == 1 else "❌"
            elif key.startswith("Amenity: "):
                aname = key.split(": ", 1)[1]
                value = "✅" if p.get("Nearby_Amenities", {}).get(aname) == 1 else "❌"
            elif key.startswith("Room Details: "):
                rdname = key.split(": ", 1)[1]
                value = p.get("Room_Details", {}).get(rdname, "N/A")
            else:
                value = p.get(key, "N/A")

            # Format some values
            if key in ["Rent_Price", "Security_Deposite"]:
                value = f"₹{value}" if value != "N/A" else value
            elif key in ["Size_In_Sqft", "Carpet_Area_Sqft"]:
                value = f"{value} sqft" if value != "N/A" else value
            elif key == "Brokerage":
                value = "Yes" if value == "yes" else "No"

            row.append(value)
        rows.append(row)

    headers = ["Attribute"] + [f"ID {p.get('property_id', 'N/A')}" for p in selected]
    print("\n" + tabulate(rows, headers=headers, tablefmt="grid"))


# --- Filtering logic ---
def filter_properties(user_input, field, data):
    filtered_properties = []
    data_field_map = {
        "size": "Size_In_Sqft", "carpet": "Carpet_Area_Sqft", "age": "Property_Age", "brokerage": "Brokerage",
        "furnishing": "Furnishing_Status", "amenities": "Number_Of_Amenities", "security": "Security_Deposite", "rent": "Rent_Price",
        "area": "Area", "zone": "Zone", "bedrooms": "Bedrooms", "bathrooms": "Bathrooms", "balcony": "Balcony",
        "floor_no": "Floor_No", "total_floors": "Total_floors_In_Building", "maintenance": "Maintenance_Charge",
        "recommended_for": "Recommended_For", "water_supply": "Water_Supply_Type", "society_type": "Society_Type",
        "road_connectivity": "Road_Connectivity", "facilities": "Facilities", "nearby_amenities": "Nearby_Amenities",
        "room_type": "Room_Details", "property_type": "Room_Details", "id": "Property_ID"
    }
    data_field = data_field_map.get(field)
    if not data_field:
        return []

    normalized_user_input = user_input.lower().strip()
    if field in ["brokerage", "furnishing", "maintenance", "recommended_for", "water_supply", "society_type"]:
        filtered_properties = [p for p in data if str(p.get(data_field, "N/A")).lower() == normalized_user_input]
    
    elif field == "facilities":
        user_facilities = [normalize_facility_name(f) for f in user_input.split(',')]
        filtered_properties = [p for p in data if all(
            normalize_facility_name(k) in user_facilities and v == 1
            for k, v in p.get("Facilities", {}).items() if k
        )]

    elif field == "nearby_amenities":
        user_amenities = [normalize_amenity_name(f) for f in user_input.split(',')]
        filtered_properties = [p for p in data if all(
            normalize_amenity_name(k) in user_amenities and v == 1
            for k, v in p.get("Nearby_Amenities", {}).items() if k
        )]

    elif field == "room_type":
        filtered_properties = [p for p in data if normalize_room_name(p.get("Room_Details", {}).get("Rooms", "")) == normalized_user_input]

    elif field == "property_type":
        filtered_properties = [p for p in data if normalize_property_type_name(p.get("Room_Details", {}).get("Type", "")) == normalized_user_input]

    elif field == "area":
        filtered_properties = [p for p in data if normalize_area_name(p.get("Area", "N/A")) == normalize_area_name(user_input)]

    elif field == "zone":
        filtered_properties = [p for p in data if normalize_zone_name(p.get("Zone", "N/A")) == normalize_zone_name(user_input)]
        
    elif field == "id":
        property_ids = [pid.strip().lower() for pid in user_input.split(",")]
        filtered_properties = [p for p in data if str(p.get("Property_ID", "")).lower() in property_ids]
    
    else:
        try:
            val = get_numeric_value(user_input)
            
            if user_input.startswith("below"):
                filtered_properties = [
                    p for p in data
                    if get_numeric_value(p.get(data_field)) is not None
                    and get_numeric_value(p.get(data_field)) < val
                ]
            elif user_input.startswith("above"):
                filtered_properties = [
                    p for p in data
                    if get_numeric_value(p.get(data_field)) is not None
                    and get_numeric_value(p.get(data_field)) > val
                ]
            elif user_input.startswith("between"):
                nums = re.findall(r"\d+", user_input)
                if len(nums) == 2:
                    low, high = int(nums[0]), int(nums[1])
                    filtered_properties = [
                        p for p in data
                        if get_numeric_value(p.get(data_field)) is not None
                        and low <= get_numeric_value(p.get(data_field)) <= high
                    ]
            else:
                filtered_properties = [
                    p for p in data
                    if get_numeric_value(p.get(data_field)) == val
                ]
        except Exception:
            return []

    return filtered_properties

# --- Format results ---
def format_property(prop):
    property_id = prop.get('property_id', 'N/A')
    rent_price = prop.get('Rent_Price', 'N/A')
    size = prop.get('Size_In_Sqft', 'Unknown')
    carpet_area = prop.get('Carpet_Area_Sqft', 'Unknown')
    security_deposit = prop.get('Security_Deposite', 'N/A')
    brokerage = prop.get('Brokerage', 'N/A')
    furnishing_status = prop.get('Furnishing_Status', 'N/A')
    amenities = prop.get('Number_Of_Amenities',0)
    age = prop.get('Property_Age', 'Unknown')
    area = prop.get('Area', 'N/A')
    zone = prop.get('Zone', 'N/A')
    bedrooms = prop.get('Bedrooms', 'N/A')
    bathrooms = prop.get('Bathrooms', 'N/A')
    balcony = prop.get('Balcony', 'N/A')
    floor_no = prop.get('Floor_No', 'N/A')
    total_floors = prop.get('Total_floors_In_Building', 'N/A')
    maintenance = prop.get('Maintenance_Charge', 'N/A')
    recommended_for = prop.get('Recommended_For', 'N/A')
    water_supply = prop.get('Water_Supply_Type', 'N/A')
    society_type = prop.get('Society_Type', 'N/A')
    road_connectivity = prop.get('Road_Connectivity', 'N/A')
    facilities_list = [k.replace("_", " ").title() for k, v in prop.get("Facilities", {}).items() if v == 1]
    facilities = ', '.join(facilities_list) if facilities_list else 'None'
    nearby_amenities_list = [k.replace("_", " ").title() for k, v in prop.get("Nearby_Amenities", {}).items() if v == 1]
    nearby_amenities = ', '.join(nearby_amenities_list) if nearby_amenities_list else 'None'
    rooms = prop.get("Room_Details", {}).get("Rooms", "N/A")
    property_type = prop.get("Room_Details", {}).get("Type", "N/A")

    return (
        f"ID: {property_id} | Rent: ₹{rent_price} | Size: {size} sqft | Carpet Area: {carpet_area} sqft\n"
        f"Rooms: {rooms} | Property Type: {property_type} | Bedrooms: {bedrooms} | Bathrooms: {bathrooms} | Balcony: {balcony}\n"
        f"Furnishing: {furnishing_status} | Security Deposit: ₹{security_deposit} | Brokerage: {brokerage}\n"
        f"Amenities: {amenities}\n"
        f"Facilities: {facilities}\n"
        f"Nearby Amenities: {nearby_amenities}\n"
        f"Floor: {floor_no}/{total_floors} | Maintenance: {maintenance} | Recommended For: {recommended_for}\n"
        f"Water Supply: {water_supply} | Society: {society_type} | Road Connectivity: {road_connectivity} km\n"
        f"Age: {age} years | Area: {area} | Zone: {zone}\n"
        f"----------------------------------------"
    )

# --- Chatbot ---
def chatbot_response():
    print("Welcome to Property Search Chatbot!")
    print("\nSearch Options:")
    print(
        "1. Size (sqft)\n2. Carpet Area (sqft)\n3. Age of Property\n4. Brokerage (yes/no)\n5. Property ID (exact match)\n"
        "6. Amenities\n7. Furnishing Status\n8. Security Deposit\n9. Rent Price\n10. Area\n11. Zone\n"
        "12. Bedrooms\n13. Bathrooms\n14. Balcony\n15. Floor Number\n16. Total Floors\n17. Maintenance Charge\n"
        "18. Recommended For\n19. Water Supply Type\n20. Society Type\n21. Road Connectivity\n22. Facilities\n"
        "23. Nearby Amenities\n24. Room Type (e.g., 1 BHK)\n25. Property Type (e.g., Flat)\n"
        "26. Compare Properties (Enter IDs separated by commas)\n"
    )
    print("💡 You can combine multiple search fields using '+'.")
    print("Example: 9+10 (Rent Price + Area)")
    print("Type 'exit' to quit.")

    CATEGORY_OPTIONS = {
    "brokerage": sorted(list(set(str(p.get("Brokerage", "N/A")).lower() for p in properties_data))),
    "furnishing": sorted(list(set(str(p.get("Furnishing_Status", "N/A")).lower() for p in properties_data))),
    "maintenance": sorted(list(set(str(p.get("Maintenance_Charge", "N/A")).lower() for p in properties_data))),
    "recommended_for": sorted(list(set(str(p.get("Recommended_For", "N/A")).lower() for p in properties_data))),
    "water_supply": sorted(list(set(str(p.get("Water_Supply_Type", "N/A")).lower() for p in properties_data))),
    "society_type": sorted(list(set(str(p.get("Society_Type", "N/A")).lower() for p in properties_data))),
    "area": ALL_AREAS,
    "zone": ALL_ZONES,
    "room_type": ALL_ROOM_TYPES,
    "property_type": ALL_PROPERTY_TYPES
    }


    search_map = {
        "1": "size", "2": "carpet", "3": "age", "4": "brokerage", "5": "id", "6": "amenities", "7": "furnishing",
        "8": "security", "9": "rent", "10": "area", "11": "zone", "12": "bedrooms", "13": "bathrooms",
        "14": "balcony", "15": "floor_no", "16": "total_floors", "17": "maintenance", "18": "recommended_for",
        "19": "water_supply", "20": "society_type", "21": "road_connectivity", "22": "facilities", "23": "nearby_amenities",
        "24": "room_type", "25": "property_type", "26": "compare"
    }
    
    while True:
        field_numbers = input("\nEnter the number(s) of search fields to combine (e.g., 9+10) or 'exit': ").strip().lower()
        if field_numbers == "exit":
            print("Goodbye!")
            break

        selected_fields = []
        is_valid_input = True
        for num in field_numbers.split('+'):
            num = num.strip()
            if num in search_map:
                selected_fields.append(search_map[num])
            else:
                print(f"❌ Invalid field number: '{num}'. Please use a valid number from the list.")
                is_valid_input = False
                break
            
        if not is_valid_input:
            continue

        if "compare" in selected_fields:
            if len(selected_fields) > 1:
                print("❌ 'Compare Properties' cannot be combined with other search options.")
                continue
            user_input = input("Enter property IDs to compare (e.g., 101, 102, 105): ").strip()
            property_ids = [pid.strip().lower() for pid in user_input.split(",")]
            if len(property_ids) < 2:
                print("⚠️ Please enter at least two Property IDs to compare.")
                continue
            compare_properties_side_by_side(properties_data, property_ids)
            continue


        combined_filters = {}
        for field in selected_fields:
            if field in CATEGORY_OPTIONS and CATEGORY_OPTIONS[field]:
                options = CATEGORY_OPTIONS[field]
                print(f"\nAvailable options for {field.title()}:")
                for idx, option in enumerate(options, start=1):
                    print(f"{idx}. {option}")
                
                choice = input(f"Enter {field} value (choose number or type value): ").strip().lower()
                
                if choice.isdigit():
                    choice_idx = int(choice) - 1
                    if 0 <= choice_idx < len(options):
                        user_input = options[choice_idx]
                    else:
                        print("❌ Invalid choice, defaulting to your raw input.")
                        user_input = choice
                else:
                    user_input = choice
            else:
                user_input = input(f"Enter {field} value: ").strip()
            
            combined_filters[field] = user_input

        results = properties_data
        for field, value in combined_filters.items():
            results = filter_properties(value, field, results)
        
        if not results:
            print("❌ No properties found matching your combined search.")
        else:
            print(f"\n✅ Found {len(results)} properties matching your combined query.")
            grouped_results = defaultdict(list)
            for prop in results:
                property_type = prop.get("Room_Details", {}).get("Type", "Other/Unspecified Type")
                grouped_results[property_type].append(prop)
            
            for prop_type, props in grouped_results.items():
                print(f"\n--- 🏠 Property Type: {str(prop_type).title()} ({len(props)} results) ---")
                for prop in props:
                    print(format_property(prop))

if __name__ == "__main__":
    chatbot_response()

In [ ]:
import json
import re
from collections import defaultdict
from tabulate import tabulate # Make sure tabulate is installed

# --- Load properties from JSON file ---
try:
    with open("property_data.json", "r") as f:
        properties_data = json.load(f)
except FileNotFoundError:
    print("Error: 'property_data.json' not found. Please ensure the file exists.")
    properties_data = []
except json.JSONDecodeError:
    print("Error: Could not decode 'property_data.json'. Please check its format.")
    properties_data = []

# --- Helper functions for normalization ---
def normalize_area_name(area_name):
    return str(area_name).replace(" ", "").lower().strip()

def normalize_zone_name(zone_name):
    return str(zone_name).replace(" ", "").lower().strip()

def normalize_facility_name(facility_name):
    return str(facility_name).replace(" ", "_").lower().strip()

def normalize_amenity_name(name):
    return str(name).replace(" ", "_").lower().strip()

def normalize_room_name(room_name):
    return str(room_name).replace(" ", "").lower().strip()

def normalize_property_type_name(prop_type):
    return str(prop_type).lower().strip()

def get_numeric_value(value):
    """Extract integer from strings like '800 sqft', '10 years', etc.
       Returns None if no number is found."""
    if not value:
        return None
    match = re.search(r"\d+", str(value))
    return int(match.group()) if match else None

# --- Dynamically get all unique values from the dataset ---
ALL_AREAS = sorted(
    list(set(normalize_area_name(p.get("Area", "N/A")) for p in properties_data))
)

ALL_ZONES = sorted(
    list(set(normalize_zone_name(p.get("Zone", "N/A")) for p in properties_data))
)

ALL_FACILITIES = []
for p in properties_data:
    if "Facilities" in p and isinstance(p["Facilities"], dict):
        ALL_FACILITIES = sorted(list(p["Facilities"].keys()))
        break

ALL_NEARBY_AMENITIES = []
for p in properties_data:
    if "Nearby_Amenities" in p and isinstance(p["Nearby_Amenities"], dict):
        ALL_NEARBY_AMENITIES = sorted(list(p["Nearby_Amenities"].keys()))
        break

ALL_ROOM_TYPES = sorted(
    list(set(normalize_room_name(p.get("Room_Details", {}).get("Rooms", "N/A"))
             for p in properties_data if p.get("Room_Details", {}).get("Rooms")))
)

ALL_PROPERTY_TYPES = sorted(
    list(set(normalize_property_type_name(p.get("Room_Details", {}).get("Type", "N/A"))
             for p in properties_data if p.get("Room_Details", {}).get("Type")))
)

# --- Comparison Function ---
def compare_properties_side_by_side(data, property_ids):
    """
    Compare multiple properties side by side in table format.
    """
    selected = [p for p in data if str(p.get("property_id", "")).lower() in property_ids]

    if not selected:
        print("⚠️ No properties found for the given IDs.")
        return

    # Collect all possible comparison keys
    comparison_keys = set()
    for p in selected:
        comparison_keys.update(p.keys())
        if isinstance(p.get("Facilities"), dict):
            comparison_keys.update([f"Facility: {k}" for k in p["Facilities"].keys()])
        if isinstance(p.get("Nearby_Amenities"), dict):
            comparison_keys.update([f"Amenity: {k}" for k in p["Nearby_Amenities"].keys()])
        # Also include nested Room_Details keys
        if isinstance(p.get("Room_Details"), dict):
            comparison_keys.update([f"Room Details: {k}" for k in p["Room_Details"].keys()])

    # Always show Property ID and Rent Price first for property comparison
    display_order = ["property_id", "Rent_Price"]
    remaining_keys = sorted(k for k in comparison_keys if k not in display_order)
    comparison_keys = display_order + remaining_keys

    # Build rows
    rows = []
    for key in comparison_keys:
        row = [key.replace('_', ' ').title()]
        for p in selected:
            value = "N/A"
            if key.startswith("Facility: "):
                fname = key.split(": ", 1)[1]
                value = "✅" if p.get("Facilities", {}).get(fname) == 1 else "❌"
            elif key.startswith("Amenity: "):
                aname = key.split(": ", 1)[1]
                value = "✅" if p.get("Nearby_Amenities", {}).get(aname) == 1 else "❌"
            elif key.startswith("Room Details: "):
                rdname = key.split(": ", 1)[1]
                value = p.get("Room_Details", {}).get(rdname, "N/A")
            else:
                value = p.get(key, "N/A")

            # Format some values
            if key in ["Rent_Price", "Security_Deposite"]:
                value = f"₹{value}" if value != "N/A" else value
            elif key in ["Size_In_Sqft", "Carpet_Area_Sqft"]:
                value = f"{value} sqft" if value != "N/A" else value
            elif key == "Brokerage":
                value = "Yes" if value == "yes" else "No"

            row.append(value)
        rows.append(row)

    headers = ["Attribute"] + [f"ID {p.get('property_id', 'N/A')}" for p in selected]
    print("\n" + tabulate(rows, headers=headers, tablefmt="grid"))


# --- Filtering logic ---
def filter_properties(user_input, field, data):
    filtered_properties = []
    data_field_map = {
        "size": "Size_In_Sqft", "carpet": "Carpet_Area_Sqft", "age": "Property_Age", "brokerage": "Brokerage",
        "furnishing": "Furnishing_Status", "amenities": "Number_Of_Amenities", "security": "Security_Deposite", "rent": "Rent_Price",
        "area": "Area", "zone": "Zone", "bedrooms": "Bedrooms", "bathrooms": "Bathrooms", "balcony": "Balcony",
        "floor_no": "Floor_No", "total_floors": "Total_floors_In_Building", "maintenance": "Maintenance_Charge",
        "recommended_for": "Recommended_For", "water_supply": "Water_Supply_Type", "society_type": "Society_Type",
        "road_connectivity": "Road_Connectivity", "facilities": "Facilities", "nearby_amenities": "Nearby_Amenities",
        "room_type": "Room_Details", "property_type": "Room_Details", "id": "Property_ID"
    }
    data_field = data_field_map.get(field)
    if not data_field:
        return []

    normalized_user_input = user_input.lower().strip()
    if field in ["brokerage", "furnishing", "maintenance", "recommended_for", "water_supply", "society_type"]:
        filtered_properties = [p for p in data if str(p.get(data_field, "N/A")).lower() == normalized_user_input]
    
    elif field == "facilities":
        user_facilities = [normalize_facility_name(f) for f in user_input.split(',')]
        filtered_properties = [p for p in data if all(
            normalize_facility_name(k) in user_facilities and v == 1
            for k, v in p.get("Facilities", {}).items() if k
        )]

    elif field == "nearby_amenities":
        user_amenities = [normalize_amenity_name(f) for f in user_input.split(',')]
        filtered_properties = [p for p in data if all(
            normalize_amenity_name(k) in user_amenities and v == 1
            for k, v in p.get("Nearby_Amenities", {}).items() if k
        )]

    elif field == "room_type":
        filtered_properties = [p for p in data if normalize_room_name(p.get("Room_Details", {}).get("Rooms", "")) == normalized_user_input]

    elif field == "property_type":
        filtered_properties = [p for p in data if normalize_property_type_name(p.get("Room_Details", {}).get("Type", "")) == normalized_user_input]

    elif field == "area":
        filtered_properties = [p for p in data if normalize_area_name(p.get("Area", "N/A")) == normalize_area_name(user_input)]

    elif field == "zone":
        filtered_properties = [p for p in data if normalize_zone_name(p.get("Zone", "N/A")) == normalize_zone_name(user_input)]
        
    elif field == "id":
        property_ids = [pid.strip().lower() for pid in user_input.split(",")]
        filtered_properties = [p for p in data if str(p.get("Property_ID", "")).lower() in property_ids]
    
    else:
        try:
            val = get_numeric_value(user_input)
            
            if user_input.startswith("below"):
                filtered_properties = [
                    p for p in data
                    if get_numeric_value(p.get(data_field)) is not None
                    and get_numeric_value(p.get(data_field)) < val
                ]
            elif user_input.startswith("above"):
                filtered_properties = [
                    p for p in data
                    if get_numeric_value(p.get(data_field)) is not None
                    and get_numeric_value(p.get(data_field)) > val
                ]
            elif user_input.startswith("between"):
                nums = re.findall(r"\d+", user_input)
                if len(nums) == 2:
                    low, high = int(nums[0]), int(nums[1])
                    filtered_properties = [
                        p for p in data
                        if get_numeric_value(p.get(data_field)) is not None
                        and low <= get_numeric_value(p.get(data_field)) <= high
                    ]
            else:
                filtered_properties = [
                    p for p in data
                    if get_numeric_value(p.get(data_field)) == val
                ]
        except Exception:
            return []

    return filtered_properties

# --- Format results ---
def format_property(prop):
    property_id = prop.get('property_id', 'N/A')
    rent_price = prop.get('Rent_Price', 'N/A')
    size = prop.get('Size_In_Sqft', 'Unknown')
    carpet_area = prop.get('Carpet_Area_Sqft', 'Unknown')
    security_deposit = prop.get('Security_Deposite', 'N/A')
    brokerage = prop.get('Brokerage', 'N/A')
    furnishing_status = prop.get('Furnishing_Status', 'N/A')
    amenities = prop.get('Number_Of_Amenities',0)
    age = prop.get('Property_Age', 'Unknown')
    area = prop.get('Area', 'N/A')
    zone = prop.get('Zone', 'N/A')
    bedrooms = prop.get('Bedrooms', 'N/A')
    bathrooms = prop.get('Bathrooms', 'N/A')
    balcony = prop.get('Balcony', 'N/A')
    floor_no = prop.get('Floor_No', 'N/A')
    total_floors = prop.get('Total_floors_In_Building', 'N/A')
    maintenance = prop.get('Maintenance_Charge', 'N/A')
    recommended_for = prop.get('Recommended_For', 'N/A')
    water_supply = prop.get('Water_Supply_Type', 'N/A')
    society_type = prop.get('Society_Type', 'N/A')
    road_connectivity = prop.get('Road_Connectivity', 'N/A')
    facilities_list = [k.replace("_", " ").title() for k, v in prop.get("Facilities", {}).items() if v == 1]
    facilities = ', '.join(facilities_list) if facilities_list else 'None'
    nearby_amenities_list = [k.replace("_", " ").title() for k, v in prop.get("Nearby_Amenities", {}).items() if v == 1]
    nearby_amenities = ', '.join(nearby_amenities_list) if nearby_amenities_list else 'None'
    rooms = prop.get("Room_Details", {}).get("Rooms", "N/A")
    property_type = prop.get("Room_Details", {}).get("Type", "N/A")

    return (
        f"ID: {property_id} | Rent: ₹{rent_price} | Size: {size} sqft | Carpet Area: {carpet_area} sqft\n"
        f"Rooms: {rooms} | Property Type: {property_type} | Bedrooms: {bedrooms} | Bathrooms: {bathrooms} | Balcony: {balcony}\n"
        f"Furnishing: {furnishing_status} | Security Deposit: ₹{security_deposit} | Brokerage: {brokerage}\n"
        f"Amenities: {amenities}\n"
        f"Facilities: {facilities}\n"
        f"Nearby Amenities: {nearby_amenities}\n"
        f"Floor: {floor_no}/{total_floors} | Maintenance: {maintenance} | Recommended For: {recommended_for}\n"
        f"Water Supply: {water_supply} | Society: {society_type} | Road Connectivity: {road_connectivity} km\n"
        f"Age: {age} years | Area: {area} | Zone: {zone}\n"
        f"----------------------------------------"
    )

# --- Chatbot ---
def chatbot_response():
    print("Welcome to Property Search Chatbot!")
    print("\nSearch Options:")
    print(
        "1. Size (sqft)\n2. Carpet Area (sqft)\n3. Age of Property\n4. Brokerage (yes/no)\n5. Property ID (exact match)\n"
        "6. Amenities\n7. Furnishing Status\n8. Security Deposit\n9. Rent Price\n10. Area\n11. Zone\n"
        "12. Bedrooms\n13. Bathrooms\n14. Balcony\n15. Floor Number\n16. Total Floors\n17. Maintenance Charge\n"
        "18. Recommended For\n19. Water Supply Type\n20. Society Type\n21. Road Connectivity\n22. Facilities\n"
        "23. Nearby Amenities\n24. Room Type (e.g., 1 BHK)\n25. Property Type (e.g., Flat)\n"
        "26. Compare Properties (Enter IDs separated by commas)\n"
    )
    print("💡 You can combine multiple search fields using '+'.")
    print("Example: 9+10 (Rent Price + Area)")
    print("Type 'exit' to quit.")

    CATEGORY_OPTIONS = {
    "brokerage": sorted(list(set(str(p.get("Brokerage", "N/A")).lower() for p in properties_data))),
    "furnishing": sorted(list(set(str(p.get("Furnishing_Status", "N/A")).lower() for p in properties_data))),
    "maintenance": sorted(list(set(str(p.get("Maintenance_Charge", "N/A")).lower() for p in properties_data))),
    "recommended_for": sorted(list(set(str(p.get("Recommended_For", "N/A")).lower() for p in properties_data))),
    "water_supply": sorted(list(set(str(p.get("Water_Supply_Type", "N/A")).lower() for p in properties_data))),
    "society_type": sorted(list(set(str(p.get("Society_Type", "N/A")).lower() for p in properties_data))),
    "area": ALL_AREAS,
    "zone": ALL_ZONES,
    "room_type": ALL_ROOM_TYPES,
    "property_type": ALL_PROPERTY_TYPES
    }


    search_map = {
        "1": "size", "2": "carpet", "3": "age", "4": "brokerage", "5": "id", "6": "amenities", "7": "furnishing",
        "8": "security", "9": "rent", "10": "area", "11": "zone", "12": "bedrooms", "13": "bathrooms",
        "14": "balcony", "15": "floor_no", "16": "total_floors", "17": "maintenance", "18": "recommended_for",
        "19": "water_supply", "20": "society_type", "21": "road_connectivity", "22": "facilities", "23": "nearby_amenities",
        "24": "room_type", "25": "property_type", "26": "compare"
    }
    
    while True:
        field_numbers = input("\nEnter the number(s) of search fields to combine (e.g., 9+10) or 'exit': ").strip().lower()
        if field_numbers == "exit":
            print("Goodbye!")
            break

        selected_fields = []
        is_valid_input = True
        for num in field_numbers.split('+'):
            num = num.strip()
            if num in search_map:
                selected_fields.append(search_map[num])
            else:
                print(f"❌ Invalid field number: '{num}'. Please use a valid number from the list.")
                is_valid_input = False
                break
            
        if not is_valid_input:
            continue

        if "compare" in selected_fields:
            if len(selected_fields) > 1:
                print("❌ 'Compare Properties' cannot be combined with other search options.")
                continue
            user_input = input("Enter property IDs to compare (e.g., 101, 102, 105): ").strip()
            property_ids = [pid.strip().lower() for pid in user_input.split(",")]
            if len(property_ids) < 2:
                print("⚠️ Please enter at least two Property IDs to compare.")
                continue
            compare_properties_side_by_side(properties_data, property_ids)
            continue


        combined_filters = {}
        for field in selected_fields:
            if field in CATEGORY_OPTIONS and CATEGORY_OPTIONS[field]:
                options = CATEGORY_OPTIONS[field]
                print(f"\nAvailable options for {field.title()}:")
                for idx, option in enumerate(options, start=1):
                    print(f"{idx}. {option}")
                
                choice = input(f"Enter {field} value (choose number or type value): ").strip().lower()
                
                if choice.isdigit():
                    choice_idx = int(choice) - 1
                    if 0 <= choice_idx < len(options):
                        user_input = options[choice_idx]
                    else:
                        print("❌ Invalid choice, defaulting to your raw input.")
                        user_input = choice
                else:
                    user_input = choice
            else:
                user_input = input(f"Enter {field} value: ").strip()
            
            combined_filters[field] = user_input

        results = properties_data
        for field, value in combined_filters.items():
            results = filter_properties(value, field, results)
        
        if not results:
            print("❌ No properties found matching your combined search.")
        else:
            print(f"\n✅ Found {len(results)} properties matching your combined query.")
            grouped_results = defaultdict(list)
            for prop in results:
                property_type = prop.get("Room_Details", {}).get("Type", "Other/Unspecified Type")
                grouped_results[property_type].append(prop)
            
            for prop_type, props in grouped_results.items():
                print(f"\n--- 🏠 Property Type: {str(prop_type).title()} ({len(props)} results) ---")
                for prop in props:
                    print(format_property(prop))

if __name__ == "__main__":
    chatbot_response()

Welcome to Property Search Chatbot!

Search Options:
1. Size (sqft)
2. Carpet Area (sqft)
3. Age of Property
4. Brokerage (yes/no)
5. Property ID (exact match)
6. Amenities
7. Furnishing Status
8. Security Deposit
9. Rent Price
10. Area
11. Zone
12. Bedrooms
13. Bathrooms
14. Balcony
15. Floor Number
16. Total Floors
17. Maintenance Charge
18. Recommended For
19. Water Supply Type
20. Society Type
21. Road Connectivity
22. Facilities
23. Nearby Amenities
24. Room Type (e.g., 1 BHK)
25. Property Type (e.g., Flat)
26. Compare Properties (Enter IDs separated by commas)

💡 You can combine multiple search fields using '+'.
Example: 9+10 (Rent Price + Area)
Type 'exit' to quit.



Enter the number(s) of search fields to combine (e.g., 9+10) or 'exit':  22
Enter facilities value:  6


❌ No properties found matching your combined search.



Enter the number(s) of search fields to combine (e.g., 9+10) or 'exit':  6
Enter amenities value:  5



✅ Found 20 properties matching your combined query.

--- 🏠 Property Type: Flat (13 results) ---
ID: Property_20 | Rent: ₹15000.0 | Size: Total Area 950.sqft  sqft | Carpet Area: Carpet Area 0.0 sqft sqft
Rooms: 2 BHK | Property Type: Flat | Bedrooms: 2 | Bathrooms: 2 | Balcony: 3
Furnishing: Semi Furnished | Security Deposit: ₹30000.0 | Brokerage: YES
Amenities: 5
Facilities: Lift, Water Supply, Kids Area, Power Backup, Parking
Nearby Amenities: Hospital, Mall, Metro Station, School
Floor: 1/3 | Maintenance: No | Recommended For: 0
Water Supply: Municipal | Society: Non-Gated | Road Connectivity: 2.0 km
Age: 5 years | Area: reshim bagh | Zone: East Zone
----------------------------------------
ID: Property_37 | Rent: ₹15000.0 | Size: Total Area 1163.sqft  sqft | Carpet Area: Carpet Area 1022.0 sqft sqft
Rooms: 2 BHK | Property Type: Flat | Bedrooms: 2 | Bathrooms: 2 | Balcony: 2
Furnishing: Semi Furnished | Security Deposit: ₹30000.0 | Brokerage: YES
Amenities: 5
Facilities: Lift, Pet

In [ ]:
import os
import speech_recognition as sr
import pyttsx3
from gtts import gTTS

# --- Supported Languages ---
LANGUAGES = {
    1: {"name": "English", "code_sr": "en-US", "code_tts": "en", "preferred_voice_id": None},
    2: {"name": "Hindi", "code_sr": "hi-IN", "code_tts": "hi", "preferred_voice_id": None},
    3: {"name": "Marathi", "code_sr": "mr-IN", "code_tts": "mr", "preferred_voice_id": None},
}

# --- Transcription Function ---
def transcribe_audio(audio_data, language_code_sr):
    recognizer = sr.Recognizer()
    try:
        text = recognizer.recognize_google(audio_data, language=language_code_sr)
        return text, None
    except sr.UnknownValueError:
        return None, "Could not understand audio."
    except sr.RequestError as e:
        return None, f"Speech recognition error: {e}"

# --- Simple Offline Chatbot Function ---
def get_offline_response(user_text, language_name):
    user_text = user_text.lower()

    if "hello" in user_text or "hi" in user_text:
        if language_name == "Hindi":
            return "नमस्ते! आप कैसे हैं?"
        elif language_name == "Marathi":
            return "नमस्कार! तुम्ही कसे आहात?"
        else:
            return "Hello! How are you?"
    elif "bye" in user_text:
        if language_name == "Hindi":
            return "अलविदा!"
        elif language_name == "Marathi":
            return "नमस्कार!"
        else:
            return "Goodbye!"
    else:
        return f"You said: {user_text}"

# --- Text-to-Speech (TTS) ---
def find_voice_by_lang(engine, lang_code_tts, preferred_voice_id=None):
    available_voices = engine.getProperty('voices')
    if preferred_voice_id:
        for voice in available_voices:
            if voice.id == preferred_voice_id:
                return voice.id
    for voice in available_voices:
        if lang_code_tts.lower() in voice.id.lower():
            return voice.id
    return None

def speak_text(text, language_code_tts, preferred_voice_id=None):
    try:
        engine = pyttsx3.init()
        voice_id = find_voice_by_lang(engine, language_code_tts, preferred_voice_id)
        if voice_id:
            engine.setProperty('voice', voice_id)
            engine.setProperty("rate", 170)
            engine.setProperty("volume", 1.0)
            engine.say(text)
            engine.runAndWait()
            engine.stop()
        else:
            # fallback to gTTS if pyttsx3 voice not available
            tts = gTTS(text=text, lang=language_code_tts)
            filename = "temp_tts.mp3"
            tts.save(filename)
            if os.name == "nt":  # Windows
                os.system(f"start {filename}")
            else:  # Linux/Mac
                os.system(f"xdg-open {filename}")
    except Exception as e:
        print(f"TTS Error: {e}")

# --- Main Loop ---
def listen_and_chat():
    recognizer = sr.Recognizer()
    mic = sr.Microphone()

    print("Please select a language:")
    for key, value in LANGUAGES.items():
        print(f"[{key}] {value['name']}")

    selected_language_code_sr = None
    selected_language_code_tts = None
    selected_language_name = None
    selected_preferred_voice_id = None

    while selected_language_code_sr is None:
        choice = input("Enter the number of your choice: ").strip()
        if choice.isdigit() and int(choice) in LANGUAGES:
            choice_int = int(choice)
            selected_language_code_sr = LANGUAGES[choice_int]['code_sr']
            selected_language_code_tts = LANGUAGES[choice_int]['code_tts']
            selected_language_name = LANGUAGES[choice_int]['name']
            selected_preferred_voice_id = LANGUAGES[choice_int]['preferred_voice_id']
            print(f"You selected {selected_language_name}.")
        else:
            print("Invalid choice.")

    print(f"🎤 Speak in {selected_language_name} (Ctrl+C to stop)...")

    while True:
        try:
            with mic as source:
                recognizer.adjust_for_ambient_noise(source)
                print("\nListening...")
                audio = recognizer.listen(source, timeout=5, phrase_time_limit=10)

            text, error = transcribe_audio(audio, selected_language_code_sr)
            if error:
                print("❌", error)
                continue

            print(f"✅ You said: {text}")
            response = get_offline_response(text, selected_language_name)
            print(f"💬 Bot: {response}")
            speak_text(response, selected_language_code_tts, selected_preferred_voice_id)

        except sr.WaitTimeoutError:
            print("⏳ Listening timeout, try again...")
        except KeyboardInterrupt:
            print("\n🛑 Stopped by user.")
            break
        except Exception as e:
            print(f"Error: {e}")

if __name__ == "__main__":
    listen_and_chat()


Please select a language:
[1] English
[2] Hindi
[3] Marathi


Enter the number of your choice:  1


You selected English.
🎤 Speak in English (Ctrl+C to stop)...

Listening...
⏳ Listening timeout, try again...

Listening...
⏳ Listening timeout, try again...

Listening...
⏳ Listening timeout, try again...

Listening...
⏳ Listening timeout, try again...
